## Overview ##

PubChem is a site run by the NIH which hosts raw data associated with chemical experiments; here we analyze the data hosted at PubChem for assay 1030, which looks for inhibitors of the protein encoding gene ALDH1A1. You can access the page for this assay [here](https://pubchem.ncbi.nlm.nih.gov/bioassay/1030)

## Results ##

We use the SMILES string, a common representation for a molecule amongst chemists, to begin the featurization process. Because the length of this string varies, it is normalized in the form of a Morgan Fingerprint; these are then used to train various binary classifiers

In [ ]:
# Exploratory data analysis and visualization

In [8]:
import pickle
import numpy as np
import pandas as pd
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem, rdMolDescriptors
from sklearn.metrics import roc_auc_score, classification_report


import warnings
warnings.filterwarnings('ignore')

global_random_state = 42
k_fold_splits = 5

np.random.seed(global_random_state)


active_pct = 0.073125471
inactive_pct = 1 - active_pct

# We set the inactive to have the weight of the active, and vice versa, to account for imbalance
class_weights = { 0: active_pct, 1: inactive_pct }

In [6]:
import keras
print(keras.backend.backend())

tensorflow


In [14]:
# What about a deep neural network?
# Sample code from: https://machinelearningmastery.com/tutorial-first-neural-network-python-keras/

from keras.models import Sequential
from keras.layers import Dense
from keras import metrics
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score, f1_score

import pickle

k_fold_splits = 2
global_random_state = 42

with open('data.classification.undersampled.pickle', 'rb') as f:
    # Pickle the 'data' dictionary using the highest protocol available.
    (X, y) = pickle.load(f)

def create_model() :
    model = Sequential()
    model.add(Dense(12, input_dim=2048, activation='relu'))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=["accuracy"])
    return model

skf = StratifiedKFold(n_splits=k_fold_splits,shuffle=True,random_state=global_random_state)

roc_auc_avg = 0
f1_score_avg = 0

for train_index, test_index in skf.split(X,y) :

    X_train = X[train_index]
    X_test = X[test_index]
    y_train = y[train_index]
    y_test = y[test_index]

    classifier = KerasClassifier(build_fn=create_model, epochs=3, batch_size=1, verbose=1)
    classifier.fit(X_train,y_train)
    y_pred = classifier.predict(X_test)
    auc = roc_auc_score(y_test, y_pred, average='macro', sample_weight=None)
    print("Computed roc_auc score of: {}".format(auc))
    print(classification_report(y_test, y_pred))
    roc_auc_avg = roc_auc_avg + auc
    
    
    y_pred_binarized = y_pred[0:] >= .4
    y_test_binarized = y_test[0:] >= .4
    fscore = f1_score(y_pred_binarized,y_test_binarized)
    print("When using regressor as an active/inactive classifier, f1 score of: {}".format(fscore))
    f1_score_avg = f1_score_avg + fscore

    
roc_auc_avg = roc_auc_avg / k_fold_splits
f1_score_avg = f1_score_avg / k_fold_splits
print("Average roc_auc score is: {}".format(roc_auc_avg))
print("Average f1_score is: {}".format(f1_score_average))

Epoch 1/3
16110/16110 [==============================] - 59s - loss: 0.6359 - acc: 0.6373    
Epoch 2/3
16110/16110 [==============================] - 59s - loss: 0.5633 - acc: 0.7094    
Epoch 3/3
16082/16112 [============================>.] - ETA: 0sComputed roc_auc score of: 0.6763282025819265
             precision    recall  f1-score   support

          0       0.67      0.70      0.68      8056
          1       0.69      0.65      0.67      8056

avg / total       0.68      0.68      0.68     16112

When using regressor as an active/inactive classifier, f1 score of: 0.6676015042386384
Epoch 1/3
16112/16112 [==============================] - 59s - loss: 0.6361 - acc: 0.6375    
Epoch 2/3
16112/16112 [==============================] - 59s - loss: 0.5643 - acc: 0.7074    
Epoch 3/3
16043/16110 [============================>.] - ETA: 0sComputed roc_auc score of: 0.6773432650527623
             precision    recall  f1-score   support

          0       0.68      0.67      0.67      

TypeError: 'numpy.float64' object is not callable

In [ ]:
# What about a deep neural network?
# Sample code from: https://machinelearningmastery.com/tutorial-first-neural-network-python-keras/

# What about a deep neural network?
# Sample code from: https://machinelearningmastery.com/tutorial-first-neural-network-python-keras/

from keras.models import Sequential
from keras.layers import Dense
from keras import metrics
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score, f1_score

import pickle

k_fold_splits = 2
global_random_state = 42

with open('data.classification.undersampled.pickle', 'rb') as f:
    # Pickle the 'data' dictionary using the highest protocol available.
    (X, y) = pickle.load(f)

def create_model() :
    model = Sequential()
    model.add(Dense(512, input_dim=2048, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=["accuracy"])
    return model

skf = StratifiedKFold(n_splits=k_fold_splits,shuffle=True,random_state=global_random_state)

roc_auc_avg = 0
f1_score_avg = 0

for train_index, test_index in skf.split(X,y) :

    X_train = X[train_index]
    X_test = X[test_index]
    y_train = y[train_index]
    y_test = y[test_index]

    classifier = KerasClassifier(build_fn=create_model, epochs=3, batch_size=1, verbose=1)
    classifier.fit(X_train,y_train)
    y_pred = classifier.predict(X_test)
    auc = roc_auc_score(y_test, y_pred, average='macro', sample_weight=None)
    print("Computed roc_auc score of: {}".format(auc))
    print(classification_report(y_test, y_pred))
    roc_auc_avg = roc_auc_avg + auc
    
    
    y_pred_binarized = y_pred[0:] >= .4
    y_test_binarized = y_test[0:] >= .4
    fscore = f1_score(y_pred_binarized,y_test_binarized)
    print("When using regressor as an active/inactive classifier, f1 score of: {}".format(fscore))
    f1_score_avg = f1_score_avg + fscore

    
roc_auc_avg = roc_auc_avg / k_fold_splits
f1_score_avg = f1_score_avg / k_fold_splits
print("Average roc_auc score is: {}".format(roc_auc_avg))
print("Average f1_score is: {}".format(f1_score_average))

Epoch 1/3
16110/16110 [==============================] - 90s - loss: 0.6471 - acc: 0.6318    
Epoch 2/3
16110/16110 [==============================] - 89s - loss: 0.5580 - acc: 0.7228    
Epoch 3/3
16062/16112 [============================>.] - ETA: 0sComputed roc_auc score of: 0.6903550148957299
             precision    recall  f1-score   support

          0       0.68      0.73      0.70      8056
          1       0.71      0.65      0.68      8056

avg / total       0.69      0.69      0.69     16112

When using regressor as an active/inactive classifier, f1 score of: 0.6786887357506279
Epoch 1/3
15015/16112 [==========================>...] - ETA: 6s - loss: 0.6493 - acc: 0.6322

In [ ]:
from keras.models import load_model
model_large = load_model("pcba_1030_large_nn.h5")
y_pred = model_large.predict_on_batch(X_test)
y_pred_binarized = y_pred[0:] > .5
print(classification_report(y_test, y_pred_binarized))